# Tarea 1: Regresión logística
Bienvenidos a la primera semana de esta especialización. Aprenderá sobre regresión logística. Concretamente, implementará regresión logística para el análisis de sentimientos en los tweets. Dado un tweet, decidirás si tiene un sentimiento positivo o negativo. Específicamente usted:

* Aprenda a extraer características para regresión logística dado algo de texto
* Implementar regresión logística desde cero
* Aplicar regresión logística en una tarea de procesamiento de lenguaje natural
* Prueba usando tu regresión logística
* Realizar análisis de errores

Usaremos un conjunto de datos de tweets. Con suerte, obtendrá una precisión superior al 99%.
Ejecute la celda a continuación para cargar los paquetes.

## Import functions and data

In [2]:
# run this cell to import nltk
import nltk
from os import getcwd

In [3]:
getcwd()

'/home/emi/Escritorio/Coursera/coursera npl/1 - Natural Language Processing with Classification and Vector Spaces/Week 1'

### Funciones importadas

Descarga los datos necesarios para esta tarea. Consulte la [documentación para el conjunto de datos twitter_samples] (http://www.nltk.org/howto/twitter.html).

* twitter_samples: si está ejecutando este cuaderno en su computadora local, deberá descargarlo usando:
`` Python
nltk.download ('twitter_samples')
''

* palabras vacías: si está ejecutando este cuaderno en su computadora local, deberá descargarlo usando:
`` pitón
nltk.download ('palabras vacías')
''

#### Importa algunas funciones auxiliares que proporcionamos en el archivo utils.py:
* `process_tweet ()`: limpia el texto, lo tokeniza en palabras separadas, elimina las palabras vacías y convierte las palabras en raíces.
* `build_freqs ()`: esto cuenta la frecuencia con la que una palabra en el 'corpus' (el conjunto completo de tweets) se asoció con una etiqueta positiva '1' o una etiqueta negativa '0', luego construye el diccionario `freqs`, donde cada clave es una tupla (palabra, etiqueta) y el valor es el recuento de su frecuencia dentro del corpus de tweets.

In [4]:
#agregue la carpeta, tmp2, desde nuestro espacio de trabajo local que contiene archivos corpora descargados previamente a la ruta de datos de nltk
# esto permite la importación de estos archivos sin descargarlos nuevamente cuando actualizamos nuestro espacio de trabajo

filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [5]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

### Prepara los datos
* El `twitter_samples` contiene subconjuntos de 5,000 tweets positivos, 5,000 tweets negativos y el conjunto completo de 10,000 tweets.
     * Si utiliza los tres conjuntos de datos, introduciríamos duplicados de los tweets positivos y negativos.
     * Seleccionará solo los cinco mil tweets positivos y los cinco mil tweets negativos.

In [6]:
# seleccione el conjunto de tweets positivos y negativos
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [7]:
# dividir los datos en dos partes, una para entrenamiento y otra para pruebas (conjunto de validación)
test_pos  = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg  = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x  = test_pos + test_neg

* Create the numpy array of positive labels and negative labels.

In [8]:
# combinar etiquetas positivas y negativas
train_y = np.append( np.ones( (len(train_pos), 1) ), np.zeros( (len(train_neg), 1) ) , axis = 0)
test_y  = np.append( np.ones( (len(test_pos) , 1) ), np.zeros( (len(test_neg) , 1) ) , axis = 0) 

In [9]:
# Imprima el tren de formas y los conjuntos de prueba
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape  = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape  = (2000, 1)


* Cree el diccionario de frecuencias usando la función `build_freqs ()` importada.
     * Recomendamos encarecidamente que abra `utils.py` y lea la función` build_freqs () `para comprender lo que está haciendo.
     * Para ver el directorio de archivos, vaya al menú y haga clic en Archivo-> Abrir.

```Python
    for y,tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
```
* Observe cómo el bucle for externo pasa por cada tweet, y el bucle for interno recorre cada palabra en un tweet.
* El diccionario `freqs` es el diccionario de frecuencias que se está construyendo.
* La clave es la tupla (palabra, etiqueta), como ("feliz", 1) o ("feliz", 0). El valor almacenado para cada clave es el recuento de cuántas veces la palabra "feliz" se asoció con una etiqueta positiva, o cuántas veces "feliz" se asoció con una etiqueta negativa.

In [10]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs)  = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs)  = 11340


#### Expected output
```
type(freqs) = <class 'dict'>
len(freqs) = 11346
```

### Tuit de proceso
La función dada "process_tweet ()" convierte el tweet en palabras individuales, elimina las palabras vacías y aplica la derivación.

In [11]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


#### Expected output
```
This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
 
This is an example of the processes version: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']
```

# Parte 1: Regresión logística


### Parte 1.1: Sigmoide
Aprenderá a utilizar la regresión logística para la clasificación de texto.
* La función sigmoidea se define como:

$$ h (z) = \frac {1} {1+ \exp ^ {- z}} \tag {1} $$

Asigna la entrada 'z' a un valor que varía entre 0 y 1, por lo que puede tratarse como una probabilidad.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='../tmp2/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> Figure 1 </div>

#### Instrucciones: Implementar la función sigmoidea
* Querrá que esta función funcione si z es un escalar así como si es una matriz.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li><a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html" > numpy.exp </a> </li>

</ul>
</p>



In [12]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def sigmoid(z): 
    '''
    entrada: z: es la entrada (puede ser un escalar o una matriz) 
    Salida:  sigmoide de z
    '''
    return 1 / (1 + np.exp(-z))

In [19]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


### Regresión logística: regresión y sigmoide

La regresión logística toma una regresión lineal regular y aplica un sigmoide a la salida de la regresión lineal.

Regresión:
$$ z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N $$
Tenga en cuenta que los valores $ \theta $ son "pesos". Si realizó la especialización en aprendizaje profundo, nos referimos a los pesos con el vector `w`. En este curso, usamos una variable diferente $ \theta $ para referirnos a los pesos.

Regresión logística
$$ h (z) = \frac {1} {1+ \exp ^ {- z}} $$
$$ z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N $$
Nos referiremos a 'z' como los 'logits'.

### Parte 1.2 Función de costo y gradiente

La función de costo utilizada para la regresión logística es el promedio de la pérdida de registro en todos los ejemplos de entrenamiento:

$$ J (\theta) = - \frac {1} {m} \sum_ {i = 1} ^ my ^ {(i)} \log (h (z (\theta) ^ {(i)})) + (1-y ^ {(i)}) \log (1-h (z (\theta) ^ {(i)})) \tag {5} $$
* $ m $ es la cantidad de ejemplos de entrenamiento
* $ y ^ {(i)} $ es la etiqueta real del i-ésimo ejemplo de entrenamiento.
* $ h (z (\theta) ^ {(i)}) $ es la predicción del modelo para el i-ésimo ejemplo de entrenamiento.

La función de pérdida para un solo ejemplo de entrenamiento es
$$ Pérdida = -1 \times \left (y ^ {(i)} \log (h (z (\theta) ^ {(i)})) + (1-y ^ {(i)}) \log (1-h (z (\theta) ^ {(i)})) \right) $$

* Todos los valores de $ h $ están entre 0 y 1, por lo que los registros serán negativos. Ésa es la razón del factor -1 aplicado a la suma de los dos términos de pérdida.
* Tenga en cuenta que cuando el modelo predice 1 ($ h (z (\theta)) = 1 $) y la etiqueta $ y $ también es 1, la pérdida para ese ejemplo de entrenamiento es 0.
* De manera similar, cuando el modelo predice 0 ($ h (z (\theta)) = 0 $) y la etiqueta real también es 0, la pérdida para ese ejemplo de entrenamiento es 0.
* Sin embargo, cuando la predicción del modelo está cerca de 1 ($ h (z (\theta)) = 0.9999 $) y la etiqueta es 0, el segundo término de la pérdida logarítmica se convierte en un número negativo grande, que luego se multiplica por el factor general de -1 para convertirlo en un valor de pérdida positivo. $ -1 \times (1 - 0) \times log (1 - 0.9999) \approx 9.2 $ Cuanto más se acerque la predicción del modelo a 1, mayor será la pérdida.

In [20]:
# Verifique que cuando el modelo predice cerca de 1, pero la etiqueta real es 0, la pérdida is a large positive value
-1 * (1 - 0) * np.log(1 - 0.9999) # la perdida es aproximadamente a  9.2

9.210340371976294

* Del mismo modo, si el modelo predice cerca de 0 ($ h (z) = 0.0001 $) pero la etiqueta real es 1, el primer término en la función de pérdida se convierte en un número grande: $ -1 \times log (0.0001) \approx 9,2 $. Cuanto más cerca de cero esté la predicción, mayor será la pérdida.

In [21]:
# Verifique que cuando el modelo predice cerca de 0 pero la etiqueta real es 1, la pérdida es un valor positivo grande
-1 * np.log(0.0001) # la perdida es aproximadamente a  9.2

9.210340371976182

#### Actualizar los pesos

Para actualizar su vector de peso $ \theta $, aplicará el descenso de gradiente para mejorar iterativamente las predicciones de su modelo.
El gradiente de la función de costo $ J $ con respecto a uno de los pesos $ \theta_j $ es:

$$ \nabla _ {\theta_j} J (\theta) = \frac {1} {m} \sum_ {i = 1} ^ m (h ^ {(i)} - y ^ {(i)}) x_j \tag {5} $$
* 'i' es el índice de todos los ejemplos de entrenamiento 'm'.
* 'j' es el índice del peso $ \theta_j $, entonces $ x_j $ es la característica asociada con el peso $  theta_j $

* Para actualizar el peso $ \theta_j $, lo ajustamos restando una fracción del gradiente determinado por $ \alpha $:
$$ \theta_j = \theta_j - \alpha \times \nabla _ {\theta_j} J (\theta) $$
* La tasa de aprendizaje $ \alpha $ es un valor que elegimos para controlar qué tan grande será una sola actualización.

## Instrucciones: Implementar la función de descenso de gradiente
* El número de iteraciones `num_iters` es el número de veces que usarás todo el conjunto de entrenamiento.
* Para cada iteración, calculará la función de costo usando todos los ejemplos de entrenamiento (hay ejemplos de entrenamiento `m`), y para todas las funciones.
* En lugar de actualizar un solo peso $\theta_i $ a la vez, podemos actualizar todos los pesos en el vector de columna:
* $ \mathbf {\theta} = \begin {pmatrix} 
\theta_0
\\
\theta_1
\\
\theta_2
\\
\vdots
\\
\theta_n
\end {pmatrix} $
* $ \mathbf {\theta} $ tiene dimensiones (n + 1, 1), donde 'n' es el número de características, y hay un elemento más para el término de sesgo $ \theta_0 $ (tenga en cuenta que el valor de característica correspondiente $ \mathbf {x_0} $ es 1).
* Los 'logits', 'z', se calculan multiplicando la matriz de características 'x' con el vector de peso 'theta'. $ z = \mathbf {x} \mathbf {\theta} $
    * $ \mathbf {x} $ tiene dimensiones (m, n + 1)
    * $ \mathbf {\theta} $: tiene dimensiones (n + 1, 1)
    * $ \mathbf {z} $: tiene dimensiones (m, 1)
* La predicción 'h', se calcula aplicando el sigmoide a cada elemento en 'z': $ h (z) = sigmoide (z) $, y tiene dimensiones (m, 1).
* La función de costo $ J $ se calcula tomando el producto escalar de los vectores 'y' y 'log (h)'. Dado que tanto 'y' como 'h' son vectores columna (m, 1), transponga el vector a la izquierda, de modo que la multiplicación matricial de un vector fila con un vector columna realice el producto escalar.
$$ J = \frac {-1} {m} \times \left (\mathbf {y} ^ T \cdot log (\mathbf {h}) + \mathbf {(1-y)} ^ T \cdot log (\mathbf {1-h}) \right) $$
* La actualización de theta también está vectorizada. Debido a que las dimensiones de $ \mathbf {x} $ son (m, n + 1), y tanto $ \mathbf {h} $ como $ \mathbf {y} $ son (m, 1), necesitamos transponer $ \mathbf {x} $ y colóquelo a la izquierda para realizar la multiplicación de matrices, que luego da la respuesta (n + 1, 1) que necesitamos:
$$ \mathbf {\theta} = \mathbf {\theta} - \frac {\alpha} {m} \times \left (\mathbf {x} ^ T \cdot \left (\mathbf {hy} \right) \right) $$

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>use np.dot para la multiplicación de matrices. </li>
     <li> Para asegurarse de que la fracción -1 / m sea un valor decimal, utilice el numerador o el denominador (o ambos), como `float (1)`, o escriba `1` para la versión flotante de 1.</li>
</ul>
</p>



In [22]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Ientrada:
         x: matriz de características que es (m, n + 1)
         y: etiquetas correspondientes de la matriz de entrada x, dimensiones (m, 1)
         theta: vector de peso de dimensión (n + 1,1)
         alfa: tasa de aprendizaje
         num_iters: número de iteraciones para las que desea entrenar su modelo
     Salida:
         J: el costo final
         theta: tu vector de peso final
     Sugerencia: es posible que desee imprimir el costo para asegurarse de que esté bajando.
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # obtener 'm', el número de filas en la matriz x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # obtener z, el producto escalar de xy theta
        z = np.dot( x , theta )
        
        # obtener el sigmoide de h
        h = sigmoid( z )
        
        # calcular la función de costo
        # tenga en cuenta que también podemos usar np.array.transpose () en lugar de np.array.T
        # np.array.T solo hace que el código sea un poco más legible :)
        J = - 1. / m * ( np.dot( y.T , np.log( h ) ) + np.dot( ( 1 - y ).T , np.log( 1 - h ) ))                                                    

        # actualizar los pesos theta
        theta = theta - (alpha / m) * np.dot( x.T , ( h - y ) )
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [23]:
# Verifica la función
# Construya un caso de prueba sintético usando numerosas funciones PRNG
np.random.seed(1)
# La entrada X es 10 x 3 con unos para los términos de sesgo
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Las etiquetas Y son 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Aplicar descenso de gradiente
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


#### Expected output
```
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]
```

## Parte 2: Extraer las características

* Dada una lista de tweets, extrae las características y guárdalas en una matriz. Extraerás dos características.
     * La primera característica es la cantidad de palabras positivas en un tweet.
     * La segunda característica es la cantidad de palabras negativas en un tweet.
* Luego entrene su clasificador de regresión logística en estas características.
* Pruebe el clasificador en un conjunto de validación.

### Instrucciones: Implementar la función extract_features.
* Esta función admite un solo tweet.
* Procese el tweet usando la función `process_tweet ()` importada y guarde la lista de palabras del tweet.
* Recorra cada palabra en la lista de palabras procesadas
     * Para cada palabra, consulte el diccionario `freqs` para el recuento cuando esa palabra tiene una etiqueta positiva '1'. (Busque la clave (palabra, 1.0)
     * Haga lo mismo para el recuento para cuando la palabra esté asociada con la etiqueta negativa '0'. (Busque la clave (palabra, 0.0).)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li> Asegúrese de manejar los casos en los que la clave (palabra, etiqueta) no se encuentra en el diccionario. </li>
     <li> Busque en la web sugerencias sobre el uso del método `.get ()` de un diccionario de Python. Aquí hay un <a href="https://www.programiz.com/python-programming/methods/dictionary/get" > example </a> </li>
</ul>
</p>


In [24]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Entrada:
         tweet: una lista de palabras para un tweet
         freqs: un diccionario correspondiente a las frecuencias de cada tupla (palabra, etiqueta)
     Salida:
         x: un vector de características de dimensión (1,3)
    '''
    # process_tweet tokeniza, deriva y elimina palabras vacías
    word_l = process_tweet(tweet)
    
    # 3 elementos en forma de vector de 1 x 3
    x = np.zeros((1, 3)) 
    
    # el término de sesgo se establece en 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # recorrer cada palabra en la lista de palabras
    for word in word_l:
        
        # incrementar el recuento de palabras para la etiqueta positiva 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # incrementar el recuento de palabras para la etiqueta negativa 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [28]:
# Check your function

# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


#### Expected output
```
[[1.00e+00 3.02e+03 6.10e+01]]
```

In [29]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


#### Expected output
```
[[1. 0. 0.]]
```

## Parte 3: Entrenamiento de su modelo

Para entrenar el modelo:
* Apile las características de todos los ejemplos de entrenamiento en una matriz `X`.
* Llame a `gradientDescent`, que ha implementado anteriormente.

Esta sección se le proporciona. Léalo para comprenderlo y ejecutar la celda.

In [30]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24216477.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]


**Expected Output**: 

```
The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]
```

# Parte 4: Pruebe su regresión logística

Es hora de que pruebe su función de regresión logística en alguna entrada nueva que su modelo no haya visto antes.

#### Instrucciones: Escribe `predict_tweet`
Predice si un tweet es positivo o negativo.

* Dado un tweet, procéselo y luego extraiga las funciones.
* Aplique los pesos aprendidos del modelo en las características para obtener los logits.
* Aplicar el sigmoide a los logits para obtener la predicción (un valor entre 0 y 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [31]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs, theta):
    '''
   Entrada:
         tuit: una cuerda
         freqs: un diccionario correspondiente a las frecuencias de cada tupla (palabra, etiqueta)
         theta: (3,1) vector de pesos
     Salida:
         y_pred: la probabilidad de que un tweet sea positivo o negativo
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [48]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))
    #print( '{:<100} -> {}'.format(tweet,predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561


**Expected Output**: 
```
I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561
```

In [33]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([[0.81636482]])

## Verifique el rendimiento usando el equipo de prueba
Después de entrenar su modelo con el conjunto de entrenamiento anterior, verifique cómo podría funcionar su modelo con datos reales no vistos probándolo con el conjunto de prueba.

#### Instrucciones: Implementar `test_logistic_regression`
* Dados los datos de prueba y los pesos de su modelo entrenado, calcule la precisión de su modelo de regresión logística.
* Utilice su función `predict_tweet ()` para hacer predicciones en cada tweet en el conjunto de prueba.
* Si la predicción es> 0,5, establezca la clasificación del modelo `y_hat` en 1; de lo contrario, establezca la clasificación del modelo` y_hat` en 0.
* Una predicción es precisa cuando `y_hat` es igual a` test_y`. Resuma todas las instancias en las que sean iguales y divida por "m".

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Use np.asarray() to convert a list to a numpy array</li>
    <li>Use np.squeeze() to make an (m,1) dimensional array into an (m,) array </li>
</ul>
</p>

In [34]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Entrada:
         test_x: una lista de tweets
         test_y: (m, 1) vector con las etiquetas correspondientes para la lista de tweets
         freqs: un diccionario con la frecuencia de cada par (o tupla)
         theta: vector de peso de dimensión (3, 1)
     Salida:
         precisión: (# de tweets clasificados correctamente) / (# total de tweets)
    """
    
    # la lista para almacenar predicciones
    y_hat = []
    
    for tweet in test_x:
        # obtener la predicción de la etiqueta para el tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # agregar 1.0 a la lista
            y_hat.append(1)
        else:
            # añadir 0 a la lista
            y_hat.append(0)

    # Con la implementación anterior, y_hat es una lista, pero test_y es (m, 1) matriz
    # convierta ambos en arreglos unidimensionales para compararlos usando el operador '=='
    
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    
    return accuracy

In [35]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


#### Expected Output: 
```0.9950```  
Pretty good!

# Parte 5: Análisis de errores

En esta parte, verá algunos tweets que su modelo clasificó incorrectamente. ¿Por qué crees que ocurrieron las clasificaciones erróneas? Específicamente, ¿qué tipo de tweets clasifica erróneamente su modelo?

In [52]:
# predict_tweet(tweet, freqs, theta) 
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('EL TWEET ES:', x)
        print('EL TWEET PROSESADO ES:', process_tweet(x))
        print()
        print('%d\t%0.8f\t%s\n' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
EL TWEET ES: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
EL TWEET PROSESADO ES: ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']

1	0.49996897	b'truli later move know queen bee upward bound movingonup'

EL TWEET ES: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
EL TWEET PROSESADO ES: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']

1	0.48650628	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'

EL TWEET ES: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
EL TWEET PROSESADO ES: ["i'm", 'play', 'brain', 'dot', 'braindot']

1	0.48370676	b"i'm play brain dot braindot"

EL TWEET ES: I'm pl

Later in this specialization, we will see how we can use deep learning to improve the prediction performance.

# Part 6: Predict with your own tweet

In [57]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[[0.48139087]]
Negative sentiment
